---
title: "Rendering a 3D Rubber Ducky using Three.js inside the Browser"
author: "Vahram Poghosyan"
date: "2024-12-26"
categories: ["Three.js", "Visualization", "JavaScript"]
format:
  html:
    css: ./css/three-js-demo.css
    code-fold: false
toc-depth: 4
jupyter: python3
highlight-style: github
include-after-body:
  text: |
    <script type="application/javascript" src="../../../javascript/light-dark.js"></script>
    <script type="importmap">
      {
          "imports": {
              "three": "https://cdn.jsdelivr.net/npm/three@0.173.0/+esm",
              "OBJLoader": "https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/OBJLoader.js",
              "MTLLoader": "https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/MTLLoader.js",
              "OrbitControls": "https://cdn.skypack.dev/three@0.133.0/examples/jsm/controls/OrbitControls.js"
          }
      }
    </script>
    <script type="module" src="./javascript/three-js-demo.js"></script>
    <script type="module" src="./javascript/three-js-grid-demo.js"></script>
    <script type="module" src="./javascript/three-js-duck-demo.js"></script>
---

# Introduction

In this post we use [Three.js](https://threejs.org/) external scripts to render 3D scenes inside this Jupyter notebook. 

This post will be very similar, in terms of its stack, to the [D3.js interactive US map post](../d3_in_jupyter_with_deno/d3_js_in_jupyter_with_deno.ipynb).

In *that* post, we learned that Canvas and SVG are two ways in which we can display complex graphics inside a web browser. We already explored SVG graphics in [D3.js interactive US map](../d3_in_jupyter_with_deno/d3_js_in_jupyter_with_deno.ipynb). It's worth noting that a lot of the same functionality could've been replicated using the HTML Canvas element instead of SVG (which we ultimately chose for its superior interactive capabilities) -- the article mentions one way to do that by using [skia-canvas](https://github.com/samizdatco/skia-canvas). 

We ended up using [linkedom](https://github.com/WebReflection/linkedom#readme) to add a DOM API on top of the Deno environment. 

In fact, what I realized later on is that we could've simply used the Markdown inside our Jupyter notebook to create the `<svg>` element without the need to introduce a third-party DOM API on top of a browser-less JavaScript environment (Deno). Then, we could have simply fetched the `TopoJSON` data and constructed the map inside our external scripts with the rest of the complex D3 animations that had to be added as external scripts. 
These scripts are run by Quarto only *after* the page has been rendered to the browser, so we can easily reference the DOM elements we create inside our notes by `class` or `id`. Crucially, the external scripts are meant to run *inside the browser* (as opposed to being pre-computed in the browser-less Deno environment). Inside the browser they're able to leverage the existing DOM API provided by the browser's engine (e.g. `document.getElementById`). Hence, this way, we eliminate the need for Deno as well as Linkedom.

We will use the [IPython.display](https://ipython.readthedocs.io/en/8.26.0/api/generated/IPython.display.html) module to create HTML elements inside our notes rather than just using Markdown directly.

# Three.js, WebGL, and Canvas API

[Three.js](https://threejs.org/) is a library for drawing 3D graphics in the browser using JavaScript and [WebGL](https://get.webgl.org/) (see [wiki](https://www.khronos.org/webgl/wiki/Main_Page)). One excellent resource for learning WebGL is the [WebGLFundamentals](https://webglfundamentals.org/webgl/lessons) website.

WebGL runs in an HTML Canvas (i.e. `<canvas>`). 

From the WebGL wiki: 

> WebGL is a DOM API, which means that it can be used in any DOM-compatible language: e.g. JavaScript

Three.js is a library that provides conveniences in JavaScript that abstract much of this [WebGL](https://developer.mozilla.org/en-US/docs/Web/API/WebGL_API) DOM API calls behind friendlier JavaScript code. 

Note that WebGL and Canvas API are two distinct ways in which the browser draws graphics, but they both require and HTML Canvas to draw on. Three.js offers a WebGL renderer as well as a Canvas renderer. Canvas API is usually a fallback option for when WebGL, the more powerful of the two APIs, isn't available. 

Note, also, that Three.js isn't suitable for modelling purposes, for that we can use [Blender](https://www.blender.org/) or a number of other closed-source 3D modelling applications. We can even download or purchase third-party models from vendors.

Canvas and SVG elements are two ways in which we can display complex graphics inside a web browser. We already explored SVG graphics in the [D3.js interactive US Map](../d3_in_jupyter_with_deno/d3_js_in_jupyter_with_deno.ipynb) post. It's worth noting that a lot of the same functionality could've been achieved by using the HTML Canvas element with D3.js instead. The post above mentions one way to do that by using [skia-canvas](https://github.com/samizdatco/skia-canvas). 

In this post we use Three.js, and not D3.js, to draw inside an HTML Canvas element using, not the Canvas API, but rather WebGL.

# 3D Scenes - A Primer

In *any* 3D scene, be it in the web browser, inside a game engine, in a movie, in a 3D modelling application, etc. there are a bunch of **geometries**, or **shapes** that are packaged with **materials** as **meshes**. The materials can describe simple properties like reflectivity, opacity, refraction index, etc. or they can be  **textures** which are just simple [raster](https://en.wikipedia.org/wiki/Raster_graphics) images. We can also apply **shaders** to our objects, which are complex mappings of pixels (or vertices) that make up an interesting animation.
 
A scene will also have one or more **light sources**, and a **camera** to *serve* the scene in some perspective.

## Meshes, Objects, Geometries, and Materials

An `object`, for the foreseeable future, means an object file of the [OBJ Wavefront format](https://en.wikipedia.org/wiki/Wavefront_.obj_file). These are files that consist of `object.children[n].geometry` and `object.children[n].material` fields. However, the material fields inside an OBJ are just *references* (via `usemtl` statements) to the true materials which come separately in `.mtl` files (typically from the same place as the `.obj` file).

There's usually no need to break the object down into individual child geometries and their corresponding objects. But, it's certainly possible. One use case would be if we want to apply a different transformation to each component. Usually, however, we load the object as a whole.

# Our First Canvas

Let's create a `<canvas>` with `id="three-d-canvas"`.

We can do it either using raw markup below this very cell, or by using the `IPython.display` module which allows us to display rich representations of objects in a Jupyter notebook (much like the `Deno.jupyter.display` module we saw in the [D3.js US map post](../d3_in_jupyter_with_deno/d3_js_in_jupyter_with_deno.ipynb)).

Using `IPython.display` is more reliable.

In [13]:
from IPython.display import display, HTML
display(HTML("<canvas id='three-d-canvas'></canvas>"))

Here's the code that produces the above output. Feel free to expand and examine. We will paint the general strokes below.

<details><summary>Click to expand the Torus code</summary>
```javascript
import * as three from 'https://cdn.jsdelivr.net/npm/three@0.173.0/+esm'

const scene = new three.Scene();

const body = document.getElementById("quarto-document-content");
const bodyWidth = body.clientWidth;
const bodyHeight = 600;

const canvas = document.getElementById("three-d-canvas")

const camera = new three.PerspectiveCamera(75, bodyWidth / bodyHeight, 0.1, 1000);
camera.position.setZ(30);

const renderer = new three.WebGLRenderer({
    canvas: canvas
});

renderer.setPixelRatio( window.devicePixelRatio );
renderer.setSize( bodyWidth, bodyHeight );


renderer.render(scene, camera);

const geometry = new three.TorusGeometry(10,3,16,100);
const material = new three.MeshBasicMaterial({ color: 0xFF6347, wireframe: true });
const torus = new three.Mesh(geometry, material);

scene.add(torus);

function animate() {
    requestAnimationFrame(animate);

    torus.rotation.x += 0.01;
    torus.rotation.y += 0.005;
    torus.rotation.z += 0.01;

    renderer.render(scene, camera);
}

animate();
```
</details>

You'll notice some general strokes.

## Creating a Scene

First, we create a `three.Scene` object.

```js
const scene = new three.Scene();
```
## Defining Scene Properties and Selecting a Canvas

Then, we define some constants for Three.js's [WebGLRenderer](https://threejs.org/docs/#api/en/renderers/WebGLRenderer) relative to the HTML document's body (grabbing the latter using the DOM API).

```js
const body = document.getElementById("quarto-document-content");
const bodyWidth = body.clientWidth;
const bodyHeight = 600;
```

We then grab the `<canvas>` element we created.

```js
const canvas = document.getElementById("three-d-canvas")
```

## Creating a Camera

After defining scene properties, we create a [PerspectiveCamera](https://threejs.org/docs/#api/en/cameras/PerspectiveCamera), supplying it the [field of view](https://en.wikipedia.org/wiki/Field_of_view) among other attributes, and setting its position.

```js
const camera = new three.PerspectiveCamera(75, bodyWidth / bodyHeight, 0.1, 1000);
camera.position.setZ(30);
```
## Creating a WebGL Renderer

We then create the `WebGLRenderer` object, supplying it the `canvas` to render, as well as setting some of its parameters to the constants we defined in step one.

```js
const renderer = new three.WebGLRenderer({
    canvas: canvas
});
renderer.setPixelRatio( window.devicePixelRatio );
renderer.setSize( bodyWidth, bodyHeight );
```

## Rendering the Scene and Adding Objects

Finally, we render the scene by invoking the renderer's `.render(scene, camera)`.

```js
renderer.render(scene, camera);
```

The `scene` object continues to be our window into the 3D world we just created. To it, we add `geometries` and their `materials` through a combination object called a [Mesh](https://threejs.org/docs/#api/en/objects/Mesh).

```js
const geometry = new three.TorusGeometry(10,3,16,100);
const material = new three.MeshBasicMaterial({ color: 0xFF6347, wireframe: true });
const torus = new three.Mesh(geometry, material);

scene.add(torus);
```

## Adding animations

We can also add a little life to the scene by using a custom animation function.
```js
function animate() {
    requestAnimationFrame(animate);

    torus.rotation.x += 0.01;
    torus.rotation.y += 0.005;
    torus.rotation.z += 0.01;

    renderer.render(scene, camera);
}
```
We invoke the animation within the outer lexical environment (for now).

```
animate();
```

# (Optional) Grid View in Jupyter Notes

To display multiple 3D `scenes` in a grid, we can simply use a CSS-grid inside Jupyter.

First we lay out the markup inside the notes:

```html
<div class="three-d-grid-container">
    <div class="three-d-grid-item"><canvas id="three-d-canvas-1"></canvas></div>
    <div class="three-d-grid-item"><canvas id="three-d-canvas-2"></canvas></div>
    <div class="three-d-grid-item"><canvas id="three-d-canvas-3"></canvas></div>
    <div class="three-d-grid-item"><canvas id="three-d-canvas-4"></canvas></div>
</div>
```

Then, we include the stylesheet in this post's subdirectory, and use Quarto's front-matter to load it into the browser (just as we load the external scripts used to produce these rich 3D outputs):

```yaml
include-after-body:
    <script type="module" src="./javascript/three-js-demo.js"></script>
```

It turns out that there's support for including CSS inside a Quarto post. It's done using the `format.html` flag in the front-matter as follows:

```yaml
format:
  html:
    css: ./css/three-js-demo.css
```

The file `three-js-demo.css` should contain these minimal styles: 

```css
#three-d-grid-container {
    display: grid;
    grid-template-columns: repeat(2, 1fr);
    grid-template-rows: repeat(2, 1fr);
    gap: 10px;
    width: 100%;
    height: 50%;
}
.three-d-grid-item {
    display: flex;
}
```

Let's add the grid markup.

Now, inside the external script, we can reference the various `canvas` elements by `id`.

In [14]:
from IPython.display import display, HTML
display(HTML(
    """
    <div id='three-d-grid-container'>
        <div class="three-d-grid-item" id="three-d-grid-item-1"><canvas id="three-d-canvas-1"></canvas></div>
        <div class="three-d-grid-item" id="three-d-grid-item-2"><canvas id="three-d-canvas-2"></canvas></div>
        <div class="three-d-grid-item" id="three-d-grid-item-3"><canvas id="three-d-canvas-3"></canvas></div>
        <div class="three-d-grid-item" id="three-d-grid-item-4"><canvas id="three-d-canvas-4"></canvas></div>
    </div>
    """
))

# Loading Custom Objects with Materials

Quarto won't serve `.obj` files (at least by default), so we can just commit the object file to the remote repository and use the `raw` GitHub link (as a CDN).

We will also need the `OBJLoader`, a Three.js add-on. This can be grabbed from a CDN as well. We may also include it in the `include-after-body.text` front matter *before* loading the script file itself as:

```yaml
<script type="importmap">
    {
        "imports": {
            "three": "https://cdn.jsdelivr.net/npm/three@0.173.0/+esm",
            "OBJLoader": "https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/OBJLoader.js"
        }
    }
</script>
```
And then, within the script, import as follows:

```js
import * as three from 'three';
import { OBJLoader } from 'OBJLoader';
```

Here's the full code to render a rubber ducky which will help us in debugging our code going forward! I downloaded this model from [Sketchfab](https://sketchfab.com/3d-models/rubber-duck-ecb9ce9ff973406398ee56e391f9c902) which hosts many such free models.

The code snippet also creates a sky background. This can be done in several ways, the most unsophisticated of which is to add a simple gradient background. 

Going one step further, we can add a skybox (a [cubemap](https://threejs.org/docs/#api/en/loaders/CubeTextureLoader)). 

```js
const loader = new three.CubeTextureLoader();
const skyboxTexture = loader.load([
  'px.jpg', // Right
  'nx.jpg', // Left
  'py.jpg', // Top
  'ny.jpg', // Bottom
  'pz.jpg', // Front
  'nz.jpg'  // Back
]);

// Set the scene's background to `skyboxTexture`
scene.background = skyboxTexture;
```

Here's a [convenient tool](https://jaxry.github.io/panorama-to-cubemap/) that lets us convert a 360$^{\circ}$ image of a sky into a cubemap. We can also download existing cubemaps from the internet. One place to get them is [Poly Haven's HDRI section](https://polyhaven.com/a/lilienstein). We can download any sky HDRs and use [this awesome tool](https://matheowis.github.io/HDRI-to-CubeMap/) which converts the HDR file into six separate textures which can be supplied to the `loader.load()` call above.

Then, [CubeCamera](https://threejs.org/docs/#api/en/cameras/CubeCamera) is used to render the skybox images. This camera exists independently of the main camera in our scene. The `CubeCamera` uses what's known as a [render target](https://webglfundamentals.org/webgl/lessons/webgl-render-to-texture.html) (a buffer where the GPU draws pixels for a scene that is being rendered in the background). Our `CubeCamera` will use [WebGLCubeRenderTarget](https://threejs.org/docs/index.html#api/en/renderers/WebGLRenderTarget) to render the skybox images.

```js
// Create the CubeRenderTarget and CubeCamera
const cubeRenderTarget = new three.WebGLCubeRenderTarget(512);
const cubeCamera = new three.CubeCamera(1, 1000, cubeRenderTarget);

// Add CubeCamera to the scene
scene.add(cubeCamera);
```

We also have the option to use the `Sky` class that's built into Three.js to generate a procedural sky. The `Sky` class is a separate import accessible at the CDN address:

```html
<script src="https://cdn.jsdelivr.net/npm/three@0.132.2/examples/js/objects/Sky.js"></script>
```
However, this is beyond the scope of this post.

Here's the final code snippet that produces the scene below.

<details><summary> Click to expand the code used to generate the rubber ducky</summary>

```js
import * as three from 'https://cdn.jsdelivr.net/npm/three@0.173.0/build/three.module.js';
import { OBJLoader } from 'https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/OBJLoader.js';
import { MTLLoader } from 'https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/MTLLoader.js';

// Get the container and set dimensions
const body = document.getElementById("quarto-document-content");
const bodyWidth = body.clientWidth;
const bodyHeight = 600;

// Set up the canvas, scene, camera, and renderer
const canvas = document.getElementById("three-d-canvas");
const scene = new three.Scene();
const camera = new three.PerspectiveCamera(75, bodyWidth / bodyHeight, 0.1, 1000);
const renderer = new three.WebGLRenderer({ canvas: canvas });
renderer.setPixelRatio(window.devicePixelRatio);
renderer.setSize(bodyWidth, bodyHeight);

// Position the camera so the object will be in view.
camera.position.set(0, 20, 0);

// Add some lights so the materials are visible.
const ambientLight = new three.AmbientLight(0xffffff, 0.6);
scene.add(ambientLight);

const directionalLight = new three.DirectionalLight(0xffffff, 0.8);
directionalLight.position.set(10, 20, 10);
scene.add(directionalLight);

// Add a skybox
const loader = new three.CubeTextureLoader();
const px = 'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/textures/skybox/px.png'
const nx = 'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/textures/skybox/nx.png'
const py = 'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/textures/skybox/py.png'
const ny = 'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/textures/skybox/ny.png'
const pz = 'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/textures/skybox/pz.png'
const nz = 'https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/textures/skybox/nz.png'
const texture = loader.load([
  px, // Right
  nx, // Left
  py, // Top
  ny, // Bottom
  pz, // Front
  nz  // Back
]);
// Set the scene's background to `texture`
scene.background = texture;
// Create the CubeRenderTarget
const cubeRenderTarget = new three.WebGLCubeRenderTarget(512);
const cubeCamera = new three.CubeCamera(1, 1000, cubeRenderTarget);
scene.add(cubeCamera);

// Instantiate MTLLoader and define the URLs for the material and object.
const mtlLoader = new MTLLoader();
const duckMtl = "https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/models/rubber_duck.mtl"
const duckObj = "https://raw.githubusercontent.com/v-poghosyan/v-poghosyan.github.io/refs/heads/main/posts/visualization/three_js_in_jupyter/models/rubber_duck.obj"

// Load the MTL file first.
mtlLoader.load(
  duckMtl,
  (materials) => {
    materials.preload();
    // Now load the OBJ file and set its materials.
    const objLoader = new OBJLoader();
    objLoader.setMaterials(materials);
    objLoader.load(
      duckObj,
      (object) => {
        // Scale and position the loaded object.
        object.scale.set(5, 5, 5);
        object.position.set(0, 15, -20);
        scene.add(object);

        // Animation loop
        function animate() {
          requestAnimationFrame(animate);
          object.rotation.y += 0.01;
          renderer.render(scene, camera);
        }
        animate();
      },
      // onProgress callback
      (xhr) => {
        console.log("Loading object...");
      },
      // onError callback
      (error) => {
        console.error('An error occurred while loading the OBJ:', error);
      }
    );
  },
  // onProgress callback for MTL
  (xhr) => {
    console.log("Loading materials...");
  },
  // onError callback for MTL
  (error) => {
    console.error('An error occurred while loading the MTL:', error);
  }
);
```
</details>

Note that `OBJLoader` takes the URL of the object followed by a callback function that gets triggered by the object fully loading. 

The geometries of the object are in `object.children[n].geometry`. Each child is a part of the object (for example the eyes, wings, and nose of the duck correspond to the object's child geometries).

Let's create another canvas to render the rubber ducky.

In [19]:
from IPython.display import display, HTML
display(HTML("<canvas id='three-d-canvas-5'></canvas>"))

For the materials, we used `MTLLoader`. Another add-on downloaded from the following CDN in the `include-after-body.text` front matter:

```yaml
<script type="importmap">
    {
        "imports": {
            "MTLLoader": "https://cdn.jsdelivr.net/npm/three@0.173.0/examples/jsm/loaders/MTLLoader.js"
        }
    }
</script>
```
Notice the nested calls? `OBJLoader` is typically called within `MTLLoader`, and gets the `material` supplied to it.

The `MTLLoader` loads the material file (`rubber_duck.mtl`) and then calls `materials.preload()`. The `OBJLoader`’s `.setMaterials(materials)` ensures that when the OBJ file is loaded, it uses the material definitions from the MTL file.

# Adding Controls

What good is rendering a skybox if we can't zoom and pan around to experience it in 3D? [OrbitControls](https://threejs.org/docs/#examples/en/controls/OrbitControls) allow us to do just that! Let's add them to our scene.

`OrbitControls` can be imported from the following CDN:

```html
<script src="https://cdn.skypack.dev/three@0.133.0/examples/jsm/controls/OrbitControls.js"></script>
```
As with all Three.js add-ons so far, we import `OrbitControls` using Quarto's front-matter.

Once it's imported, we add `OrbitControls` to our scene.

```js
// Add OrbitControls
const controls = new OrbitControls(camera, renderer.domElement);
controls.enableDamping = true; // Smooth damping effect during rotation
controls.dampingFactor = 0.05;
```
The line `const controls = new OrbitControls(camera, renderer.domElement)` passes `renderer.domElement` instead of just `renderer` because `OrbitControls` needs a DOM element to attach event listeners to (for mouse or touch interactions), not a renderer object. A Three.js rendering object manages the WebGL context, `renderer.domElement` is the actual HTML Canvas element to which Three.js renders the scene.

Furthermore, inside the `animate()` call, we need to ensure that we're updating `CubeCamera`'s environment map. This is done to retain realistic reflections on shiny objects. Reflective objects show surrounding environment reflected on their surfaces, so the `CubeMap` needs to be updated whenever the scene changes in order to keep reflections on objects accurate.

The animate block, which previously looked like:
```js
function animate() {
  requestAnimationFrame(animate);
  object.rotation.y += 0.01;
  renderer.render(scene, camera);
}
```
Becomes:
```js
function animate() {
  requestAnimationFrame(animate);
  object.rotation.y += 0.01;
  cubeCamera.update(renderer, scene);  
  renderer.render(scene, camera);
  controls.update();
}
```
The line `conrols.update()` is required the animation loop to ensure smooth interaction and rendering. Without this call, the camera might not respond to user input correctly.

Also, now that we have `OrbitControls`, let's ensure we're positioning the camera to show the duck at the start of our scene. This is done within the `OBJLoader` callback, where the `object` is defined.

```js
camera5.position.copy(object.position);  // Set camera at duck position
camera5.position.y += 20;  // Add y-offset (adjust value as needed)
camera5.position.z += -20  // Add z-offset offset (adjust value as needed)
camera5.lookAt(object.position) // Orient the camera to look at the duck
```

Go ahead and try the controls in the canvas above. Use:

* Right mouse button (RMB) to pan
* Left mouse button (LMB) to rotate
* Scroll wheel (SW) to zoom

Before we move on to creating more complex scenes, with multiple objects, we should explore how an OBJ file represents an object. We will need to understand the structure of an Object to manipulate it correctly using Three.js. 

We will do that in an upcoming update!